## **Federated Learning for attack detection: 5 nodes sharing weights**

IDs from this file = **id10xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Transform to One Hot Encoding the Categories - normal, attack
    data_labels.insert(0, 'normal', data['attack_cat'].replace('normal', 1).replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'attack', data['attack_cat'].replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 1).replace('normal', 0))

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 5A

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part5.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/3207952841.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/3207952841.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/3207952841.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/3207952841.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or s

In [6]:
# Define 
node_datasets = [training1, training2, training3, training4, training5]
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1000.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [11]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1000.hdf5')

Epoch 1/5
31/31 [==============================] - 7s 214ms/step - loss: 0.3943 - accuracy: 0.9653 - val_loss: 1.7129 - val_accuracy: 0.6194
Epoch 2/5
31/31 [==============================] - 6s 202ms/step - loss: 0.0643 - accuracy: 0.9916 - val_loss: 1.1418 - val_accuracy: 0.6200
Epoch 3/5
31/31 [==============================] - 6s 191ms/step - loss: 0.0306 - accuracy: 0.9922 - val_loss: 0.6976 - val_accuracy: 0.6345
Epoch 4/5
31/31 [==============================] - 6s 191ms/step - loss: 0.0272 - accuracy: 0.9922 - val_loss: 0.7039 - val_accuracy: 0.6326
Epoch 5/5
31/31 [==============================] - 6s 193ms/step - loss: 0.0258 - accuracy: 0.9923 - val_loss: 0.7505 - val_accuracy: 0.6254
Epoch 1/5
31/31 [==============================] - 7s 237ms/step - loss: 0.0444 - accuracy: 0.9902 - val_loss: 0.3993 - val_accuracy: 0.8214
Epoch 2/5
31/31 [==============================] - 6s 191ms/step - loss: 0.0284 - accuracy: 0.9911 - val_loss: 0.6198 - val_accuracy: 0.6391
Epoch 3/5
31/

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 5s 3ms/step - loss: 0.1920 - accuracy: 0.9093
Epoch 1/5
31/31 [==============================] - 5s 168ms/step - loss: 0.0160 - accuracy: 0.9951 - val_loss: 0.4438 - val_accuracy: 0.7691
Epoch 2/5
31/31 [==============================] - 5s 166ms/step - loss: 0.0157 - accuracy: 0.9952 - val_loss: 0.3325 - val_accuracy: 0.8242
Epoch 3/5
31/31 [==============================] - 5s 166ms/step - loss: 0.0147 - accuracy: 0.9956 - val_loss: 0.3442 - val_accuracy: 0.8231
Epoch 4/5
31/31 [==============================] - 5s 165ms/step - loss: 0.0145 - accuracy: 0.9955 - val_loss: 0.4058 - val_accuracy: 0.8101
Epoch 5/5
31/31 [==============================] - 5s 165ms/step - loss: 0.0144 - accuracy: 0.9956 - val_loss: 0.3537 - val_accuracy: 0.8305
Epoch 1/5
31/31 [==============================] - 5s 164ms/step - loss: 0.0160 - accuracy: 0.9951 - val_loss: 0.5204 - val_accuracy: 0.7834
Epoch 2/5
31/31 [==============================] - 5s 175ms/ste

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 5s 3ms/step - loss: 0.5650 - accuracy: 0.7394
Epoch 1/5
31/31 [==============================] - 5s 177ms/step - loss: 0.0136 - accuracy: 0.9959 - val_loss: 0.4749 - val_accuracy: 0.8116
Epoch 2/5
31/31 [==============================] - 5s 167ms/step - loss: 0.0132 - accuracy: 0.9962 - val_loss: 0.5764 - val_accuracy: 0.7809
Epoch 3/5
31/31 [==============================] - 5s 168ms/step - loss: 0.0133 - accuracy: 0.9960 - val_loss: 0.4218 - val_accuracy: 0.8215
Epoch 4/5
31/31 [==============================] - 5s 166ms/step - loss: 0.0132 - accuracy: 0.9961 - val_loss: 0.4770 - val_accuracy: 0.8054
Epoch 5/5
31/31 [==============================] - 5s 165ms/step - loss: 0.0128 - accuracy: 0.9963 - val_loss: 0.4961 - val_accuracy: 0.8016
Epoch 1/5
31/31 [==============================] - 5s 175ms/step - loss: 0.0141 - accuracy: 0.9959 - val_loss: 0.4294 - val_accuracy: 0.8367
Epoch 2/5
31/31 [==============================] - 6s 182ms/ste

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 5s 3ms/step - loss: 0.8705 - accuracy: 0.6733


In [12]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06492394208908081, 0.19202971458435059], [0.09822872281074524, 0.5650112628936768], [0.12339755147695541, 0.870532751083374]]
Accuracy for iterations:  [[0.9684770107269287, 0.9093220829963684], [0.9629261493682861, 0.7393736839294434], [0.9589383006095886, 0.6733088493347168]]
F1 for iterations:  [[0.9684224244475296, 0.9079446594003465], [0.962810208743622, 0.7114403379229214], [0.9587823563071344, 0.6161749438257738]]
Precision for iterations:  [[0.9692894589894216, 0.9173583051340704], [0.9649416043187354, 0.8103986817351979], [0.9615910415422185, 0.7809228810934163]]
Recall for iterations:  [[0.9684770224078997, 0.9093220954733706], [0.9629261737123492, 0.7393736830632857], [0.9589383271494931, 0.6733088716122938]]


#### Dataset 5B 

In [13]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/4261673179.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/4261673179.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/4261673179.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/4261673179.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or s

In [14]:
# Define 
node_datasets = [training1, training2, training3, training4, training5]
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [15]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1001.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [16]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [17]:
global_model = build_model((24,1,1))

In [18]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [19]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1001.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 162ms/step - loss: 0.3388 - accuracy: 0.9771 - val_loss: 2.1843 - val_accuracy: 0.5518
Epoch 2/5
31/31 [==============================] - 5s 167ms/step - loss: 0.0505 - accuracy: 0.9916 - val_loss: 0.9944 - val_accuracy: 0.5540
Epoch 3/5
31/31 [==============================] - 5s 174ms/step - loss: 0.0291 - accuracy: 0.9923 - val_loss: 0.8833 - val_accuracy: 0.5641
Epoch 4/5
31/31 [==============================] - 5s 166ms/step - loss: 0.0272 - accuracy: 0.9922 - val_loss: 0.9497 - val_accuracy: 0.5583
Epoch 5/5
31/31 [==============================] - 5s 165ms/step - loss: 0.0260 - accuracy: 0.9924 - val_loss: 0.7349 - val_accuracy: 0.5745
Epoch 1/5
31/31 [==============================] - 5s 174ms/step - loss: 0.0424 - accuracy: 0.9903 - val_loss: 0.5374 - val_accuracy: 0.7160
Epoch 2/5
31/31 [==============================] - 6s 190ms/step - loss: 0.0271 - accuracy: 0.9911 - val_loss: 0.6042 - val_accuracy: 0.6419
Epoch 3/5
31/

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 5s 3ms/step - loss: 0.1647 - accuracy: 0.9489
Epoch 1/5
31/31 [==============================] - 5s 168ms/step - loss: 0.0210 - accuracy: 0.9930 - val_loss: 0.4809 - val_accuracy: 0.6696
Epoch 2/5
31/31 [==============================] - 5s 167ms/step - loss: 0.0187 - accuracy: 0.9941 - val_loss: 0.6008 - val_accuracy: 0.6298
Epoch 3/5
31/31 [==============================] - 5s 167ms/step - loss: 0.0173 - accuracy: 0.9947 - val_loss: 0.3377 - val_accuracy: 0.7944
Epoch 4/5
31/31 [==============================] - 6s 180ms/step - loss: 0.0160 - accuracy: 0.9949 - val_loss: 0.4260 - val_accuracy: 0.7585
Epoch 5/5
31/31 [==============================] - 5s 168ms/step - loss: 0.0157 - accuracy: 0.9949 - val_loss: 0.2794 - val_accuracy: 0.8558
Epoch 1/5
31/31 [==============================] - 5s 172ms/step - loss: 0.0169 - accuracy: 0.9948 - val_loss: 0.4057 - val_accuracy: 0.7968
Epoch 2/5
31/31 [==============================] - 5s 168ms/ste

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 0.0971 - accuracy: 0.9765
Epoch 1/5
31/31 [==============================] - 6s 196ms/step - loss: 0.0158 - accuracy: 0.9951 - val_loss: 0.5332 - val_accuracy: 0.7211
Epoch 2/5
31/31 [==============================] - 6s 187ms/step - loss: 0.0145 - accuracy: 0.9956 - val_loss: 0.4215 - val_accuracy: 0.7797
Epoch 3/5
31/31 [==============================] - 6s 187ms/step - loss: 0.0141 - accuracy: 0.9959 - val_loss: 0.3998 - val_accuracy: 0.7997
Epoch 4/5
31/31 [==============================] - 5s 170ms/step - loss: 0.0143 - accuracy: 0.9959 - val_loss: 0.3743 - val_accuracy: 0.8153
Epoch 5/5
31/31 [==============================] - 5s 172ms/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 0.3279 - val_accuracy: 0.8434
Epoch 1/5
31/31 [==============================] - 5s 172ms/step - loss: 0.0148 - accuracy: 0.9957 - val_loss: 0.6375 - val_accuracy: 0.7549
Epoch 2/5
31/31 [==============================] - 5s 168ms/ste

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 5s 3ms/step - loss: 0.1286 - accuracy: 0.9639


In [20]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.0867944061756134, 0.16470888257026672], [0.03516291826963425, 0.0971357449889183], [0.037969958037137985, 0.12858761847019196]]
Accuracy for iterations:  [[0.9491001963615417, 0.9488846659660339], [0.9909433722496033, 0.9765130877494812], [0.9855166673660278, 0.9639250040054321]]
F1 for iterations:  [[0.9488842740056337, 0.9486127452561498], [0.9909463198861936, 0.9765008641808393], [0.9855160795159222, 0.9638348278908137]]
Precision for iterations:  [[0.9521007166111072, 0.950541977371563], [0.9910302059642497, 0.9765217688534996], [0.9855164440206748, 0.9643572623430751]]
Recall for iterations:  [[0.9491001782114581, 0.9488846908377534], [0.9909433521283123, 0.9765131148732108], [0.9855166671535832, 0.96392501634818]]


#### Dataset 5C

In [21]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_6960/4154259209.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part5.csv')


In [22]:
# Define 
node_datasets = [training1, training2, training3, training4, training5]
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [23]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1002.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [24]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [25]:
global_model = build_model((24,1,1))

In [26]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [27]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1002.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 179ms/step - loss: 0.5617 - accuracy: 0.7300 - val_loss: 1.6020 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 161ms/step - loss: 0.2339 - accuracy: 0.9235 - val_loss: 3.3507 - val_accuracy: 0.1288
Epoch 3/5
16/16 [==============================] - 2s 155ms/step - loss: 0.0957 - accuracy: 0.9893 - val_loss: 4.2276 - val_accuracy: 0.1337
Epoch 4/5
16/16 [==============================] - 3s 160ms/step - loss: 0.0593 - accuracy: 0.9893 - val_loss: 2.2942 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 157ms/step - loss: 0.0388 - accuracy: 0.9898 - val_loss: 1.3347 - val_accuracy: 0.1800
Epoch 1/5
14/14 [==============================] - 2s 172ms/step - loss: 0.0226 - accuracy: 0.9950 - val_loss: 0.7685 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 2s 157ms/step - loss: 0.0205 - accuracy: 0.9950 - val_loss: 0.5687 - val_accuracy: 0.7290
Epoch 3/5

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 0.1513 - accuracy: 0.9573
Epoch 1/5
16/16 [==============================] - 3s 157ms/step - loss: 0.0307 - accuracy: 0.9901 - val_loss: 0.8628 - val_accuracy: 0.4910
Epoch 2/5
16/16 [==============================] - 3s 174ms/step - loss: 0.0266 - accuracy: 0.9900 - val_loss: 1.4338 - val_accuracy: 0.1564
Epoch 3/5
16/16 [==============================] - 2s 153ms/step - loss: 0.0246 - accuracy: 0.9906 - val_loss: 1.1253 - val_accuracy: 0.2912
Epoch 4/5
16/16 [==============================] - 3s 156ms/step - loss: 0.0232 - accuracy: 0.9915 - val_loss: 0.8781 - val_accuracy: 0.4636
Epoch 5/5
16/16 [==============================] - 2s 155ms/step - loss: 0.0219 - accuracy: 0.9921 - val_loss: 1.0077 - val_accuracy: 0.3904
Epoch 1/5
14/14 [==============================] - 2s 170ms/step - loss: 0.0167 - accuracy: 0.9953 - val_loss: 0.7141 - val_accuracy: 0.7278
Epoch 2/5
14/14 [==============================] - 2s 165ms/ste

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 5s 3ms/step - loss: 0.1169 - accuracy: 0.9717
Epoch 1/5
16/16 [==============================] - 3s 171ms/step - loss: 0.0268 - accuracy: 0.9915 - val_loss: 1.2463 - val_accuracy: 0.3406
Epoch 2/5
16/16 [==============================] - 3s 193ms/step - loss: 0.0204 - accuracy: 0.9931 - val_loss: 0.9053 - val_accuracy: 0.4993
Epoch 3/5
16/16 [==============================] - 3s 159ms/step - loss: 0.0191 - accuracy: 0.9936 - val_loss: 1.0327 - val_accuracy: 0.4528
Epoch 4/5
16/16 [==============================] - 3s 172ms/step - loss: 0.0187 - accuracy: 0.9937 - val_loss: 0.8801 - val_accuracy: 0.5398
Epoch 5/5
16/16 [==============================] - 3s 159ms/step - loss: 0.0183 - accuracy: 0.9940 - val_loss: 0.9063 - val_accuracy: 0.5400
Epoch 1/5
14/14 [==============================] - 2s 177ms/step - loss: 0.0144 - accuracy: 0.9962 - val_loss: 0.6795 - val_accuracy: 0.7426
Epoch 2/5
14/14 [==============================] - 2s 160ms/ste

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 3ms/step - loss: 0.1252 - accuracy: 0.9666: 0s - loss: 0.1247 - accuracy: 0.


In [28]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.09872888773679733, 0.15131351351737976], [0.12024590373039246, 0.11685054004192352], [0.12895263731479645, 0.12522338330745697]]
Accuracy for iterations:  [[0.978322446346283, 0.957313060760498], [0.9688568115234375, 0.9716994762420654], [0.9641166925430298, 0.9666497111320496]]
F1 for iterations:  [[0.978335505113202, 0.9573955484948163], [0.9688825098598914, 0.9717691391251647], [0.9641447047900135, 0.9667394851144581]]
Precision for iterations:  [[0.9787138383022526, 0.958142008093737], [0.9707612324360201, 0.9730941427738602], [0.9666378442992711, 0.968435550001543]]
Recall for iterations:  [[0.9783224750942183, 0.9573130858097798], [0.9688568173186479, 0.9716994841241009], [0.964116684682579, 0.9666497129986195]]


### Experimenting with epochs and iterations 

Use of dataset 5A, considering it contains an equal partition of the data between the 5 nodes. 

#### #Epochs = 10, #Iterations = 10

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part5.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3, training4, training5]
num_nodes = 5
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1010.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1010.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### #Epochs = 5, #Iterations = 5

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-5A-Part5.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3, training4, training5]
num_nodes = 5
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1011.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1011.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)